In [ ]:
# 기사 목록 

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # 이미지 로드 비활성화
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 검색어, 시작 날짜, 종료 날짜 입력
keywords = [
    "검색어",
]
start_date = "2024-06-01"
end_date = "2024-08-31"


collected_urls = set()  

# 크롬 드라이버 설정
driver = webdriver.Chrome(options=chrome_options)

def search_and_save_articles(keyword, start_date, end_date):
    base_url = "https://www.jejunews.com/engine_yonhap/search.php?page="
    results = []


    page = 1
    while True:
        print(f"{keyword} {page} 수집 중...")
        search_url = base_url + f"{page}&picktab=article&searchcont=article&others_cont_type=&div_code=&cust_div_code=&sfield=&article_type=&period=all&from_date={start_date}&to_date={end_date}&sort=date&searchword={urllib.parse.quote(keyword)}"
        
        driver.get(search_url)
        
        attempts = 0
        while attempts < 3:
            try:
                WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.contents")))
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                articles = soup.find_all('div', class_='list')
                if articles:
                    break  # 기사가 있으면 루프 종료
            except Exception as e:
                print(f"페이지 로딩 중 오류 발생: {e}")
            attempts += 1
            if attempts < 3:
                print(f"기사가 안 뜨면 새로고침 시도 {attempts}번")
                driver.refresh()  # 새로고침

        # 기사 목록 가져오기        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles = soup.find_all('div', class_='list')
        print(f"찾은 기사 수: {len(articles)}")  # 로그 추가

        if not articles:
            print("기사를 찾을 수 없습니다.")
            break

        new_articles_found = False  # 새로운 기사가 발견되었는지 확인하는 플래그

        # 기사 정보 추출
        for article in articles:
            try:
                title_tag = article.select_one('div.title strong')
                link_tag = article.select_one('div.title a')
                info_tags = article.select('ul.info li')
                
                title = title_tag.get_text(strip=True) if title_tag else ""
                url = link_tag['href'] if link_tag else ""
                writer = ""
                date = ""
                
                if len(info_tags) == 3:
                    writer = info_tags[1].get_text(strip=True)
                    date = info_tags[2].get_text(strip=True)
                elif len(info_tags) == 2:
                    writer = ""
                    date = info_tags[1].get_text(strip=True)
                else:
                    writer = ""
                    date = ""


                if url and url not in collected_urls:
                    collected_urls.add(url)
                    new_articles_found = True  # 새로운 기사가 발견됨
                    # 결과 리스트에 추가
                    results.append({
                        'media': '제주일보', 
                        'title': title, 
                        'date': date, 
                        'section': '', 
                        'article_type': '', 
                        'writer': writer, 
                        'page': '', 
                        'url': url, 
                        'content': ''  # 컨텐츠 정보 없음
                    })
                    print(f"기사 추가: {title}, {url}")  # 로그 추가
            except Exception as e:
                print(f"기사 처리 중 오류 발생: {e}")

        if not new_articles_found:  # 새로운 기사가 발견되지 않으면 종료
            print("더 이상 새로운 기사가 없습니다.")
            break

        # 다음 페이지로 이동
        page += 1
                          
    return results

def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)

all_results = []

try:
    for keyword in keywords:
        all_results.extend(search_and_save_articles(keyword, start_date, end_date))

    df = pd.DataFrame(all_results)

    # 기사 수 출력
    print(f"Total number of articles: {len(df)}")

    save_to_excel(df, f'2제주일보 {start_date}~{end_date}.xlsx')

finally:
    # 드라이버 종료
    driver.quit()


In [ ]:
# 본문 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# subtitle 컬럼이 없을 경우 추가
if 'sub-title' not in df.columns:
    df['sub-title'] = ""  # 서브 타이틀 컬럼 추가

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문을 스크랩하는 함수 정의
def get_content_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 서브 타이틀 추출
        sub_tit = soup.find('div', class_='article-head-sub')  # 서브 타이틀 추출
        article_body = soup.find('div', id='article-view-content-div')

        if article_body:
            # 서브 타이틀이 본문에 포함되어 있으면 제거
            if sub_tit:
                subtitle = sub_tit.get_text(separator='\n', strip=True)
                sub_tit.decompose()
            else:
                subtitle = ""

            # 'caption' 클래스를 가진 모든 태그를 제거합니다.
            for caption in article_body.find_all('figure'):
                caption.decompose()

            # 'font-size:13px;padding:8px 10px 7px;color:#666;background:#f5f5f5;' 스타일을 가진 태그 제거
            for element in article_body.find_all('figcaption'):
                element.decompose()

            # 'emoji-for' div 이후의 모든 내용을 제거
            emoji_div = article_body.find('div',class_='text-center')
            if emoji_div:
                # emoji-for div를 포함하여 그 이후 모든 형제 요소를 제거
                for sibling in emoji_div.find_next_siblings():
                    sibling.decompose()
                emoji_div.decompose()  # emoji-for div 자체도 제거


            # p 태그로 나누어진 본문이 있는 경우와 없는 경우 처리
            paragraphs = article_body.find_all('p')
            if paragraphs:
                # p 태그가 있을 경우 p 태그 간에는 한 줄 띄우고, 내부 태그들은 엔터 없이 연결
                content = '\n'.join(''.join(inner.get_text(strip=True) for inner in p.find_all(text=True)) for p in paragraphs)
            else:
                # p 태그가 없을 경우 전체 텍스트를 가져옴
                content = article_body.get_text(separator='\n', strip=True)
        else:
            content = ""
            subtitle = ""

        return content, subtitle
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문 내용을 스크랩하여 content와 sub-title 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_from_url(row['url'])
    if content is None:
        error_log.append(row['url'])
    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle  # 서브 타이틀 추가

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 오류 로그 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")
